In [1]:
import mixsol as mx

Create a list of `Powder` objects, each referring to a solid reagent that can be used to create target `Solution`s

In [2]:
powders = [
    mx.Powder('Cs_I'), #if only using elements, you can simply input the formula delimited by underscores
    mx.Powder({'Pb':1, 'I':2}), #you can also use a dictionary to specify the components
    mx.Powder('Pb_Br2'),
    mx.Powder('Pb_Cl2'),
    mx.Powder(
        formula='MA_I', #you can have components that are not element (like methylammonium/MA here), but then the molar mass must be specified
        molar_mass=mx.calculate_molar_mass('C_H6_N_I'), #molar mass can either be given as a float value, or the helper function can take an elemental formula (as string or dict) and calculate the molar mass
        alias='MAI', #you can also give an alias for the powder - this just changes the string representation of the powder when printed
    ),
    mx.Powder(
        formula='FA_I',
        molar_mass = mx.calculate_molar_mass('C_H5_N2_I'),
        alias='FAI',
        )
]

The list of available `Powder`'s is used to initialize a `Weigher` object, which can calculate the masses of powders required to make desired solutions

In [3]:
weigher = mx.Weigher(
    powders=powders
)

We can define a target `Solution`, then get the masses of our powders required to create that solution. Keep in mind that all volume/mass units are in SI units (liters, grams)

In [4]:
target=mx.Solution(
    solutes='Cs0.05_FA0.8_MA0.15_Pb_I2.4_Br0.45_Cl0.15',
    solvent='DMF9_DMSO1',
    molarity=1
)

masses = weigher.get_weights(
    target,
    volume=100e-6, #in L
)
print(masses) #masses of each powder, in grams

{'Cs_I': 0.00129904961, 'Pb461_I922': 7e-05, 'Pb_Br2': 0.0082576575, 'Pb_Cl2': 0.0020857935, 'MAI': 0.002384543385, 'FAI': 0.01375746568}


We can also go the other way -- given a dictionary of masses of powders and a volume of solvent, the resulting `Solution` can be computed

In [5]:
result = weigher.weights_to_solution(
    weights=masses, #the dictionary of masses we just created
    volume=100e-6, #volume of solution (L) 
    solvent='DMF9_DMSO1', #solvent system
)
print(result)

2.4M Cs0.0208_Br0.187_Cl0.0625_I_FA0.333_Pb0.417_MA0.0625 in DMF0.9_DMSO0.1


In [6]:
result == target #we made it back to our target solution!

True

Note that while the string representation of the result `Solution` looks quite different from the original target `Solution`, the two are still equivalent. The issue is that the molarity of the resulting solution is not known, and by default is set to the amount of the component of highest molarity.

We can control this by setting the molarity when computing the result `Solution`, though note that this is only for aesthetics - under the hood, the total moles of each component will be unchanged. molarity can be set in three ways: by a set value, by a single component, or by a list of components to which the molarity will be normalized

In [7]:
result_fixedmolarity = weigher.weights_to_solution(
    weights=masses,
    volume=100e-6,
    solvent='DMF9_DMSO1',
    molarity=1
)
print(result_fixedmolarity)

1M Cs0.05_Br0.45_Cl0.15_I2.4_FA0.8_Pb1_MA0.15 in DMF0.9_DMSO0.1


In [8]:
result_singlecomponent = weigher.weights_to_solution(
    weights=masses,
    volume=100e-6,
    solvent='DMF9_DMSO1',
    molarity="Pb"
)
print(result_singlecomponent)

1M Cs0.05_Br0.45_Cl0.15_I2.4_FA0.8_Pb_MA0.15 in DMF0.9_DMSO0.1


In [16]:
result_multicomponent = weigher.weights_to_solution(
    weights=masses,
    volume=100e-6,
    solvent='DMF9_DMSO1',
    molarity=["I", "Br", "Cl"]
)
print(result_multicomponent)

3M Cs0.0167_Br0.15_Cl0.05_I0.8_FA0.267_Pb0.333_MA0.05 in DMF0.9_DMSO0.1


In [21]:
result == result_singlecomponent == result_multicomponent == result_fixedmolarity #all methods give the same result!

True